<a href="https://colab.research.google.com/github/iraidaantropova/data-processing-method/blob/main/%D0%9F%D0%A05_%D0%A1%D0%B8%D0%BD%D0%B8%D1%86%D0%B0%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание

1 вариант

1. Доработать паука в имеющимся проекте, чтобы он формировал item по структуре

-наименование вакансии

-зарплата от

-зарплата до

-ссылку на саму вакансию

2. создать в имеющимся проекте второго паука по сбору вакансий с superjob


2 вариант

1. Создать пауков по сбору данных о книгах с сайтов labirint.ru и/или book24.ru

2. Каждый паук должен собирать:

- ссылку на книгу

- наименование книги

- автор

- основную цену

- рейтинг книги

3. собранная информация должна складываться в базу данных

Вариант 2

In [45]:
pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy.crawler import CrawlerProcess

In [47]:
class BookparserLabirintItem(scrapy.Item):
    "Bookparser item"
    title = scrapy.Field()
    link = scrapy.Field()
    author = scrapy.Field()
    own_price = scrapy.Field()
    sale_price = scrapy.Field()
    rating = scrapy.Field()

In [48]:
class LabirintruSpider(scrapy.Spider):
      name = 'labirintru'
      allowed_domains = ['labirint.ru'] 
      start_urls = ['https://www.labirint.ru/']


In [49]:
def parse (self, response):
    links = response.xpath('//a[@class="product-title-link"]/@href').extract()
    for link in links:
        yield response.follow('https://www.labirint.ru' + link, callback=self.parse_item)
    next_page = response.xpath('//a[@class="pagination-next__text"]/@href').extract_first()
    if next_page:
        yield response.follow('https://www.labirint.ru/novelty/' + next_page, callback=self.parse)

In [50]:
def parse_item(self, response:HtmlResponse):
        title = response.xpath('//h1/text()').extract_first()
        link = response.url
        author = response.xpath('//a[@data-event-label="author"]/text()').extract_first()
        own_price = response.xpath('//span[@class="buying-price-val-number"]/text()').extract_first()
        if not own_price:
            own_price = response.xpath('//span[@class="buying-priceold-val-number"]/text()').extract_first()
        sale_price = response.xpath('//span[@class="buying-pricenew-val-number"]/text()').extract_first()
        rating = response.xpath('//div[@id="rate"]/text').extract_first()

        yield BookparserLabirintItem(title=title, link=link, author=author, own_price=own_price, sale_price=sale_price,
                                     rating=rating)